In [2]:
import os
import glob
import fnmatch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from src.configs import configs

In [3]:
epoc = pd.read_spss(f"{configs.PROJECT_ROOT}/EPOC/EPOC_data_forSP9_mLPP.sav")

NameError: name 'pd' is not defined

In [173]:
epoc.head()

,Code,Code2,Familiencode,Relation,Status,StatusOCD_binary,Center,Gruppe_Neu,Sample_verlaufBerlin,Finish_vsQuit,...,Cz_400to1000_Av_pos_sweet_GC,Cz_400to1000_Av_pos_affiliation_GC,Cz_400to1000_Av_neg_GC,Cz_400to1000_Av_neg_human_GC,Cz_400to1000_Av_neg_nature_GC,Cz_400to1000_Av_neg_weapons_GC,Cz_400to1000_Av_neutr_GC,Cz_400to1000_Av_neutr_living_GC,Cz_400to1000_Av_neutr_items_GC,Cz_400to1000_Av_neutr_environs_GC
0,epoc_p_1112,EPOC_P_1112,3196.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,reguläres Therapieende,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,epoc_p_1064,EPOC_P_1064,3128.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,reguläres Therapieende,...,2.407063,3.327511,5.650495,8.410168,2.938624,5.536354,1.214722,2.711010,2.457612,-1.477090
2,epoc_p_1118,EPOC_P_1118,3208.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,reguläres Therapieende,...,1.433177,0.518778,5.951720,0.995575,2.376250,14.205409,-4.424898,-3.708006,-4.273278,-5.553788
3,epoc_p_1050,EPOC_P_1050,3094.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,Therapieabbruch,...,2.677901,-2.315895,0.443861,-3.399431,1.612670,3.196841,-2.968737,-4.438069,-0.721836,-3.652792
4,epoc_p_1069,EPOC_P_1069,3133.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,reguläres Therapieende,...,-0.221830,1.038309,2.689956,0.585481,6.851810,0.681768,-0.506852,1.956024,-1.882689,-1.629634


In [174]:
epoc.shape

(304, 342)

#### Check variable values

In [5]:
epoc['Status'].unique()

['Index', 'Kontrolle']
Categories (2, object): ['Index', 'Kontrolle']

In [6]:
epoc['Ausschluss'].unique()

['Kein Ausschluss', 'Ausschluss']
Categories (2, object): ['Ausschluss', 'Kein Ausschluss']

In [7]:
epoc['Response'].value_counts(dropna=False)

NaN    179
1.0     75
0.0     50
Name: Response, dtype: int64

In [8]:
epoc['Remission'].value_counts(dropna=False)

NaN    179
0.0     63
1.0     62
Name: Remission, dtype: int64

#### Delete healthy controls

In [175]:
target_to_list_status = ['Index']

epoc = epoc[epoc['Status'].isin(target_to_list_status)]
print(epoc.shape)

(142, 342)


#### Delete "Ausschluss" subjects

In [176]:
target_to_list = ['Kein Ausschluss']

epoc = epoc[epoc['Ausschluss'].isin(target_to_list)]
print(epoc.shape)

(134, 342)


#### Delete subjects with missing "Response" or "Remission" values

In [177]:
epoc = epoc.dropna(subset=['Response'])
print(epoc.shape)

(119, 342)


In [178]:
epoc = epoc.dropna(subset=['Remission'])
print(epoc.shape)

(119, 342)


Subjects with missing Response values are identical to subjects with missing Remission values

#### Check for NaNs

In [13]:
epoc['Geschlecht'].value_counts(dropna=False)

Weiblich    62
Männlich    57
Name: Geschlecht, dtype: int64

In [14]:
epoc['Alter'].value_counts(dropna=False)

30.0    7
28.0    7
36.0    7
29.0    6
26.0    6
23.0    6
31.0    6
38.0    5
18.0    5
27.0    5
33.0    5
32.0    4
41.0    4
20.0    4
24.0    4
35.0    3
37.0    3
34.0    3
22.0    3
50.0    3
25.0    2
55.0    2
19.0    2
48.0    2
52.0    2
21.0    2
42.0    2
51.0    2
45.0    2
39.0    1
44.0    1
49.0    1
46.0    1
61.0    1
Name: Alter, dtype: int64

#### Save files for all patients (that were not excluded from the study)

In [15]:
# all variables
epoc.to_csv(f'{configs.PROJECT_ROOT}/EPOC/csv_files/EPOC_all_pat_119.csv', index=False)

#### Save files for patients that have structural MRI scans

In [17]:
pat_labels_julia = epoc['Code'].unique()
pat_labels_julia.sort()
print(pat_labels_julia)
print(pat_labels_julia.shape)

['epoc_p_1001' 'epoc_p_1004' 'epoc_p_1005' 'epoc_p_1006' 'epoc_p_1008'
 'epoc_p_1010' 'epoc_p_1011' 'epoc_p_1012' 'epoc_p_1013' 'epoc_p_1014'
 'epoc_p_1015' 'epoc_p_1016' 'epoc_p_1017' 'epoc_p_1018' 'epoc_p_1019'
 'epoc_p_1021' 'epoc_p_1022' 'epoc_p_1023' 'epoc_p_1024' 'epoc_p_1025'
 'epoc_p_1027' 'epoc_p_1028' 'epoc_p_1029' 'epoc_p_1030' 'epoc_p_1031'
 'epoc_p_1032' 'epoc_p_1033' 'epoc_p_1034' 'epoc_p_1035' 'epoc_p_1036'
 'epoc_p_1037' 'epoc_p_1038' 'epoc_p_1039' 'epoc_p_1040' 'epoc_p_1042'
 'epoc_p_1043' 'epoc_p_1044' 'epoc_p_1045' 'epoc_p_1046' 'epoc_p_1047'
 'epoc_p_1048' 'epoc_p_1049' 'epoc_p_1050' 'epoc_p_1051' 'epoc_p_1052'
 'epoc_p_1053' 'epoc_p_1054' 'epoc_p_1055' 'epoc_p_1056' 'epoc_p_1057'
 'epoc_p_1059' 'epoc_p_1060' 'epoc_p_1061' 'epoc_p_1062' 'epoc_p_1063'
 'epoc_p_1064' 'epoc_p_1065' 'epoc_p_1067' 'epoc_p_1068' 'epoc_p_1069'
 'epoc_p_1071' 'epoc_p_1072' 'epoc_p_1073' 'epoc_p_1074' 'epoc_p_1077'
 'epoc_p_1079' 'epoc_p_1080' 'epoc_p_1081' 'epoc_p_1082' 'epoc_p_1084'
 'epoc

In [18]:
pat_labels_julia = np.ndarray.tolist(pat_labels_julia)

In [19]:
pat_labels_christian = fnmatch.filter(os.listdir(f'{configs.PROJECT_ROOT}/EPOC/EPOC_BIDS/EPOC_derivatives'), 'sub-epocp*')
pat_labels_christian.sort()
pat_labels_christian

['sub-epocp1004',
 'sub-epocp1005',
 'sub-epocp1006',
 'sub-epocp1008',
 'sub-epocp1009',
 'sub-epocp1012',
 'sub-epocp1013',
 'sub-epocp1014',
 'sub-epocp1015',
 'sub-epocp1016',
 'sub-epocp1017',
 'sub-epocp1018',
 'sub-epocp1019',
 'sub-epocp1021',
 'sub-epocp1022',
 'sub-epocp1023',
 'sub-epocp1024',
 'sub-epocp1025',
 'sub-epocp1026',
 'sub-epocp1027',
 'sub-epocp1028',
 'sub-epocp1029',
 'sub-epocp1030',
 'sub-epocp1031',
 'sub-epocp1033',
 'sub-epocp1034',
 'sub-epocp1035',
 'sub-epocp1036',
 'sub-epocp1037',
 'sub-epocp1039',
 'sub-epocp1040',
 'sub-epocp1041',
 'sub-epocp1042',
 'sub-epocp1043',
 'sub-epocp1044',
 'sub-epocp1045',
 'sub-epocp1046',
 'sub-epocp1047',
 'sub-epocp1048',
 'sub-epocp1049',
 'sub-epocp1050',
 'sub-epocp1051',
 'sub-epocp1052',
 'sub-epocp1053',
 'sub-epocp1054',
 'sub-epocp1056',
 'sub-epocp1057',
 'sub-epocp1059',
 'sub-epocp1061',
 'sub-epocp1062',
 'sub-epocp1063',
 'sub-epocp1064',
 'sub-epocp1065',
 'sub-epocp1066',
 'sub-epocp1067',
 'sub-epoc

In [20]:
pat_labels_christian_new = []
for subj in pat_labels_christian:
    
    subj_new = subj[4:8] + '_' + subj[8] + '_' + subj[9:None]
    pat_labels_christian_new.append(subj_new)

In [21]:
len(pat_labels_christian_new)

103

In [22]:
epoc_T1only = epoc.loc[epoc['Code'].isin(pat_labels_christian_new)]

In [23]:
epoc_T1only.shape

(90, 342)

In [24]:
epoc_T1only

,Code,Code2,Familiencode,Relation,Status,StatusOCD_binary,Center,Gruppe_Neu,Sample_verlaufBerlin,Finish_vsQuit,...,Cz_400to1000_Av_pos_sweet_GC,Cz_400to1000_Av_pos_affiliation_GC,Cz_400to1000_Av_neg_GC,Cz_400to1000_Av_neg_human_GC,Cz_400to1000_Av_neg_nature_GC,Cz_400to1000_Av_neg_weapons_GC,Cz_400to1000_Av_neutr_GC,Cz_400to1000_Av_neutr_living_GC,Cz_400to1000_Av_neutr_items_GC,Cz_400to1000_Av_neutr_environs_GC
1,epoc_p_1064,EPOC_P_1064,3128.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,reguläres Therapieende,...,2.407063,3.327511,5.650495,8.410168,2.938624,5.536354,1.214722,2.711010,2.457612,-1.477090
2,epoc_p_1118,EPOC_P_1118,3208.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,reguläres Therapieende,...,1.433177,0.518778,5.951720,0.995575,2.376250,14.205409,-4.424898,-3.708006,-4.273278,-5.553788
3,epoc_p_1050,EPOC_P_1050,3094.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,Therapieabbruch,...,2.677901,-2.315895,0.443861,-3.399431,1.612670,3.196841,-2.968737,-4.438069,-0.721836,-3.652792
4,epoc_p_1069,EPOC_P_1069,3133.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,reguläres Therapieende,...,-0.221830,1.038309,2.689956,0.585481,6.851810,0.681768,-0.506852,1.956024,-1.882689,-1.629634
6,epoc_p_1043,EPOC_P_1043,3077.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,reguläres Therapieende,...,23.685087,12.619618,12.989751,7.685421,15.390666,16.024837,1.628977,10.361793,-3.753685,-1.953933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,epoc_p_1017,EPOC_P_1017,3040.0,Index mit Verwandten,Index,1.0,Berlin,OCD_NoMed,1.0,reguläres Therapieende,...,6.732655,7.212780,11.968004,13.530252,10.949705,11.241917,3.389360,4.948088,2.101350,3.228076
123,epoc_p_1049,EPOC_P_1049,3093.0,Index mit Verwandten,Index,1.0,Berlin,OCD_NoMed,1.0,reguläres Therapieende,...,6.763960,-0.702261,5.329402,2.991900,8.288413,4.581462,0.409690,2.269908,-4.352084,3.311585
124,epoc_p_1108,EPOC_P_1108,3190.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_NoMed,1.0,reguläres Therapieende,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,epoc_p_1016,EPOC_P_1016,3036.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_NoMed,1.0,reguläres Therapieende,...,1.301927,3.592755,-0.419727,-0.360400,3.465178,-4.128371,0.252107,-0.527917,-2.170540,3.001500


In [25]:
epoc_T1only.head()

,Code,Code2,Familiencode,Relation,Status,StatusOCD_binary,Center,Gruppe_Neu,Sample_verlaufBerlin,Finish_vsQuit,...,Cz_400to1000_Av_pos_sweet_GC,Cz_400to1000_Av_pos_affiliation_GC,Cz_400to1000_Av_neg_GC,Cz_400to1000_Av_neg_human_GC,Cz_400to1000_Av_neg_nature_GC,Cz_400to1000_Av_neg_weapons_GC,Cz_400to1000_Av_neutr_GC,Cz_400to1000_Av_neutr_living_GC,Cz_400to1000_Av_neutr_items_GC,Cz_400to1000_Av_neutr_environs_GC
1,epoc_p_1064,EPOC_P_1064,3128.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,reguläres Therapieende,...,2.407063,3.327511,5.650495,8.410168,2.938624,5.536354,1.214722,2.711010,2.457612,-1.477090
2,epoc_p_1118,EPOC_P_1118,3208.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,reguläres Therapieende,...,1.433177,0.518778,5.951720,0.995575,2.376250,14.205409,-4.424898,-3.708006,-4.273278,-5.553788
3,epoc_p_1050,EPOC_P_1050,3094.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,Therapieabbruch,...,2.677901,-2.315895,0.443861,-3.399431,1.612670,3.196841,-2.968737,-4.438069,-0.721836,-3.652792
4,epoc_p_1069,EPOC_P_1069,3133.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,reguläres Therapieende,...,-0.221830,1.038309,2.689956,0.585481,6.851810,0.681768,-0.506852,1.956024,-1.882689,-1.629634
6,epoc_p_1043,EPOC_P_1043,3077.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,reguläres Therapieende,...,23.685087,12.619618,12.989751,7.685421,15.390666,16.024837,1.628977,10.361793,-3.753685,-1.953933


In [26]:
# all variables
epoc_T1only.to_csv(f'{configs.PROJECT_ROOT}/EPOC/csv_files/EPOC_T1_pat_90.csv', index=False)

#### Save file for patients that have rs-fMRI scans

In [28]:
pat_labels_fMRI = fnmatch.filter(os.listdir(f'{configs.PROJECT_ROOT}/EPOC/halfpipe_patients_only'), 'sub-epocp*')
pat_labels_fMRI.sort()
pat_labels_fMRI

['sub-epocp1004',
 'sub-epocp1005',
 'sub-epocp1006',
 'sub-epocp1008',
 'sub-epocp1009',
 'sub-epocp1012',
 'sub-epocp1013',
 'sub-epocp1014',
 'sub-epocp1015',
 'sub-epocp1016',
 'sub-epocp1017',
 'sub-epocp1018',
 'sub-epocp1019',
 'sub-epocp1021',
 'sub-epocp1022',
 'sub-epocp1023',
 'sub-epocp1024',
 'sub-epocp1025',
 'sub-epocp1026',
 'sub-epocp1027',
 'sub-epocp1029',
 'sub-epocp1030',
 'sub-epocp1031',
 'sub-epocp1033',
 'sub-epocp1034',
 'sub-epocp1035',
 'sub-epocp1036',
 'sub-epocp1037',
 'sub-epocp1039',
 'sub-epocp1040',
 'sub-epocp1041',
 'sub-epocp1043',
 'sub-epocp1044',
 'sub-epocp1045',
 'sub-epocp1047',
 'sub-epocp1048',
 'sub-epocp1050',
 'sub-epocp1051',
 'sub-epocp1052',
 'sub-epocp1054',
 'sub-epocp1057',
 'sub-epocp1059',
 'sub-epocp1061',
 'sub-epocp1062',
 'sub-epocp1063',
 'sub-epocp1064',
 'sub-epocp1065',
 'sub-epocp1066',
 'sub-epocp1067',
 'sub-epocp1069',
 'sub-epocp1070',
 'sub-epocp1071',
 'sub-epocp1072',
 'sub-epocp1073',
 'sub-epocp1074',
 'sub-epoc

In [29]:
pat_labels_fMRI_new = []
for subj in pat_labels_fMRI:
    
    subj_new = subj[4:8] + '_' + subj[8] + '_' + subj[9:None]
    pat_labels_fMRI_new.append(subj_new)

In [30]:
len(pat_labels_fMRI_new)

93

In [31]:
epoc_fMRI = epoc.loc[epoc['Code'].isin(pat_labels_fMRI_new)]

In [32]:
epoc_fMRI.shape

(80, 342)

In [33]:
epoc_fMRI.head()

,Code,Code2,Familiencode,Relation,Status,StatusOCD_binary,Center,Gruppe_Neu,Sample_verlaufBerlin,Finish_vsQuit,...,Cz_400to1000_Av_pos_sweet_GC,Cz_400to1000_Av_pos_affiliation_GC,Cz_400to1000_Av_neg_GC,Cz_400to1000_Av_neg_human_GC,Cz_400to1000_Av_neg_nature_GC,Cz_400to1000_Av_neg_weapons_GC,Cz_400to1000_Av_neutr_GC,Cz_400to1000_Av_neutr_living_GC,Cz_400to1000_Av_neutr_items_GC,Cz_400to1000_Av_neutr_environs_GC
1,epoc_p_1064,EPOC_P_1064,3128.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,reguläres Therapieende,...,2.407063,3.327511,5.650495,8.410168,2.938624,5.536354,1.214722,2.711010,2.457612,-1.477090
2,epoc_p_1118,EPOC_P_1118,3208.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,reguläres Therapieende,...,1.433177,0.518778,5.951720,0.995575,2.376250,14.205409,-4.424898,-3.708006,-4.273278,-5.553788
3,epoc_p_1050,EPOC_P_1050,3094.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,Therapieabbruch,...,2.677901,-2.315895,0.443861,-3.399431,1.612670,3.196841,-2.968737,-4.438069,-0.721836,-3.652792
4,epoc_p_1069,EPOC_P_1069,3133.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,reguläres Therapieende,...,-0.221830,1.038309,2.689956,0.585481,6.851810,0.681768,-0.506852,1.956024,-1.882689,-1.629634
6,epoc_p_1043,EPOC_P_1043,3077.0,Index ohne Verwandte,Index,1.0,Berlin,OCD_Med,1.0,reguläres Therapieende,...,23.685087,12.619618,12.989751,7.685421,15.390666,16.024837,1.628977,10.361793,-3.753685,-1.953933


In [34]:
epoc_fMRI.to_csv(f'{configs.PROJECT_ROOT}/EPOC/csv_files/EPOC_fMRI_pat_80.csv', index=False)